In [3]:
import pandas as pd
from config import get_configs
from gmf import GMFEngine
from mlp import MLPEngine
from cnn import CNNEngine
from neumf import NeuMFEngine
from data import SampleGenerator

model = "gmf"
data = "ratings.csv"

# Load Data
print("load data")
data = pd.read_csv(f"./data/processed/{data}")

print(f"Range of userId is [{data.userId.min()}, {data.userId.max()}]")
print(f"Unique userIds {data.userId.nunique()}")

print(f"Range of itemId is [{data.itemId.min()}, {data.itemId.max()}]")
print(f"Unique itemIds {data.itemId.nunique()}")

print(f"Range of rating is [{data.rating.min()}, {data.rating.max()}]")
print(f"Unique ratings {data.rating.nunique()}")

num_users = data.userId.max() + 1
num_items = data.itemId.max() + 1

# DataLoader for training
print("initialise sample generator")
sample_generator = SampleGenerator(ratings=data)
evaluate_data = sample_generator.evaluate_data

# Specify the exact model
print("start traing model")

configs = get_configs(num_users, num_items)

config, engine = None, None
if model == "gmf":
    config = configs["gmf_config"]
    engine = GMFEngine(config)
elif model == "mlp":
    config = configs["mlp_config"]
    engine = MLPEngine(config)
elif model == "cnn":
    config = configs["cnn_config"]
    engine = CNNEngine(config)
elif model == "neumf":
    config = configs["neumf_config"]
    engine = NeuMFEngine(config)

assert config != None, "No model chosen for training"
assert engine != None, "No model chosen for training"

for epoch in range(config["num_epoch"]):
    print("Epoch {} starts".format(epoch))
    print("-" * 80)
    train_loader = sample_generator.instance_a_train_loader(
        config["num_negative"], config["batch_size"]
    )
    engine.train_an_epoch(train_loader, epoch_id=epoch)
    hit_ratio, ndcg = engine.evaluate(evaluate_data, epoch_id=epoch)

    if epoch == config["num_epoch"] - 1:
        engine.save(config["alias"], epoch, hit_ratio, ndcg)

load data
Range of userId is [0, 6039]
Unique userIds 6040
Range of itemId is [0, 3705]
Unique itemIds 3706
Range of rating is [1, 5]
Unique ratings 5
initialise sample generator
start traing model
GMF(
  (embedding_user): Embedding(6040, 8)
  (embedding_item): Embedding(3706, 8)
  (affine_output): Linear(in_features=8, out_features=1, bias=True)
  (logistic): Sigmoid()
)
Epoch 0 starts
--------------------------------------------------------------------------------
[Training Epoch 0] Batch 0, Loss 0.1780375987291336
[Training Epoch 0] Batch 1, Loss 0.17347502708435059
[Training Epoch 0] Batch 2, Loss 0.18350522220134735
[Training Epoch 0] Batch 3, Loss 0.18533121049404144
[Training Epoch 0] Batch 4, Loss 0.18095162510871887
[Training Epoch 0] Batch 5, Loss 0.18043309450149536
[Training Epoch 0] Batch 6, Loss 0.180410236120224
[Training Epoch 0] Batch 7, Loss 0.17861086130142212
[Training Epoch 0] Batch 8, Loss 0.1760123074054718
[Training Epoch 0] Batch 9, Loss 0.17859286069869995
[Tr

KeyboardInterrupt: 